In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
%%capture
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [5]:
import torch
import clip
from PIL import Image
from itertools import islice
import glob
import re

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 59.7MiB/s]


In [7]:
def get_prediction(frame_path, list_of_labels, how_many_predictions) -> list:
    Highest3Predictions = []
    na = frame_path.split("/")
    name = na[len(na) - 1]
    try:
        text = clip.tokenize(list_of_labels).to(device)
        image = preprocess(Image.open(frame_path)).unsqueeze(0).to(device)
        with torch.no_grad():
            logits_per_image, logits_per_text = model(image, text)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()
            probs = probs.tolist()[0]
        vv = {}
        for i, j in enumerate(probs):
            vv[list_of_labels[i]] = j
        maxx = {k: v for k, v in sorted(vv.items(), key=lambda item: item[1], reverse=True)}
        Highest3Predictions = list(islice(maxx.items(), how_many_predictions))
        print(f"{name}")
        c1 = Highest3Predictions[0][0]
        s1 = round(100 * Highest3Predictions[0][1], 2)
        c2 = Highest3Predictions[1][0]
        s2 = round(100 * Highest3Predictions[1][1], 2)
        c3 = Highest3Predictions[2][0]
        s3 = round(100 * Highest3Predictions[2][1], 2)
        print(c1, s1)
        print(c2, s2)
        print(c3, s3)
        print("___________________________")
    except:
        raise

    return Highest3Predictions

In [8]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [atoi(c) for c in re.split(r'(\d+)', text)]

In [9]:
how_many_predictions = 3

In [10]:
first_level = {
    # Nothing on the screen black blank blur
    "image showing an empty or unoccupied space": "neutral",
    "image showing a still or blank or uneventful background or black background": "neutral",
    "image showing passive or monotonous or insignificant event": "neutral",
    "image out of focus or blurry due to motion or camera shake": "neutral",
    "image showing a unrecognized activity": "neutral",
    "image showing a close-up shot of unrecognized object": "neutral",
    "image showing red color, red spread visible in the background, red spread on the walls, floor and ceiling ": "neutral",
    # Credits, Warning, watermark, logo, production logo
    "image showing centered words or credits on a screen": "neutral",
    "image showing sentences written in the middle of the screen": "neutral",
    "image showing statutory warning written on the screen": "neutral",
    "image showing a movie production company or movie studio logo or movie title still": "neutral",
    "image showing a brand logo or service logo or trademark logo or organization logo or a product logo": "neutral",
    # Cropped parts
    "image showing a person's face with only the top half visible": "neutral",
    "image showing a landscape with only the bottom portion showing": "neutral",
    "image showing a building with the top and sides cut off": "neutral",
    "image showing a car with only the front or back visible": "neutral",
    "image showing a person's body with only the feet or head visible": "neutral",
    "image showing a flower with only the center or petals showing": "neutral",
    "image showing a tree with only the trunk or top branches visible": "neutral",
    "image showing a food dish with only a portion of the plate visible": "neutral",
    "image showing a sports field with only a portion of the playing area visible": "neutral",
    "image showing a computer or electronic device with only part of the screen or keyboard visible": "neutral",
    "image showing a cityscape with only a portion of the skyline visible": "neutral",
    "image showing a beach with only a portion of the water or sand visible": "neutral",
    "image showing a mountain with only a portion of the peak or base showing": "neutral",
    "image showing a street with only a portion of the buildings visible.": "neutral",
    "image showing individual body with hands or legs visible": "neutral",
    "image showing a couple body with some part of body visible": "neutral",
    # Objects
    "image showing water bottles or water containers": "neutral",
    "image showing fruit juice bottles or cola tins": "neutral",
    "image showing cola tins or cola bottles, soft drinks": "neutral",
    "image showing a cabinet full with soft drink bottles": "neutral",
    "image showing a close up shot of a red dress or clothe": "neutral",
    "image showing a brown dog standing in a grassy field": "neutral",
    "image showing a red apple on a wooden table": "neutral",
    "image showing a woman wearing a blue shirt and holding a yellow umbrella": "neutral",
    "image showing a car parked on a city street": "neutral",
    "image showing a black and white cat sitting on a windowsill": "neutral",
    "image showing a bowl of vegetable soup with pieces of carrot and broccoli visible": "neutral",
    "image showing a tall tree with green leaves in a park": "neutral",
    "image showing a laptop computer on a desk": "neutral",
    "image showing a man wearing glasses and reading a book in a library": "neutral",
    "image showing a group of people walking on a sandy beach with waves in the background": "neutral",
    "image showing a red and yellow hot air balloon floating in a clear blue sky": "neutral",
    "image showing a mug filled with steaming coffee on a wooden table": "neutral",
    "image showing a row of colorful tulips in a garden": "neutral",
    "image showing a horse grazing in a green pasture": "neutral",
    "image showing a person jogging on a city street": "neutral",
    "image showing a bowl of ripe strawberries with green leaves on a white plate": "neutral",
    "image showing a taxi driving down a busy city street at night.": "neutral",
    "image showing a modern kitchen with stainless steel appliances and a granite counter top": "neutral",
    "image showing a young child sitting on a swing in a playground": "neutral",
    "image showing a group of tourists taking photos of a historic monument": "neutral",
    "image showing a bowl of hot and spicy noodles with green onions on top": "neutral",
    "image showing a black and white photograph of a city skyline at night": "neutral",
    "image showing a woman wearing a pink dress and sunglasses sitting at an outdoor cafe": "neutral",
    "image showing a large, colorful kite flying high in the sky": "neutral",
    "image showing a classic red bicycle leaning against a brick wall": "neutral",
    "image showing a close-up of a yellow daffodil with dew drops on the petals": "neutral",
    "image showing a white sailboat floating on calm blue waters": "neutral",
    "image showing a pile of books with colorful spines on a wooden bookshelf": "neutral",
    "image showing a pair of headphones on a background": "neutral",
    "image showing a stack of pancakes with butter and syrup on a plate": "neutral",
    "image showing a man in a blue shirt and jeans walking a dog": "neutral",
    "image showing a purple and yellow sunset over a calm lake": "neutral",
    "image showing a bouquet of red roses in a glass vase on a wooden table": "neutral",
    "image showing a group of people sitting around a campfire at night": "neutral",
    "image showing a white swan swimming on a lake with trees in the background": "neutral",
    "image showing a vintage red telephone booth on a busy city street": "neutral",
    "image showing a green park bench in front of a large fountain": "neutral",
    "image showing a woman wearing a black and white striped shirt holding a bouquet of pink flowers.": "neutral",
    "image showing a slice of pepperoni pizza on a white plate": "neutral",
    "image showing close-up of hand fist": "neutral",
    "image showing red velvet clothe": "neutral",
    "image showing a person in red, blood colored clothes": "neutral",
    "image showing a dirty face": "neutral",
    # Vehicles
    "image showing a person driving a car": "neutral",
    # People, Group, Person:
    "image showing group of men and women enjoying themselves, engaging in a activity": "neutral",
    "image showing group of people engaged in a local gathering": "neutral",
    "image showing gathering of group of people, people walking": "neutral",
    # close up Shots
    "image showing a side view of a person's face": "neutral",
    "image showing a close up shot of a person's face": "neutral",
    "image showing a close up shot of a person's limp arm": "neutral",
    # long shots
    "image showing a person with a clean face with open mouth, showing drops of sweat on the face": "neutral",
    "image showing an orgasm expression on the face": "neutral",
    # Animal:
    "image showing animal roaming freely in its natural habitat": "neutral",
    "image showing a human petting or holding an animal affectionately": "neutral",
    "image showing humans playing with animals": "neutral",
    "image showing animal enjoying themselves": "neutral",
    "image showing animal mating or mating habit": "neutral",
    # Couple activities
    "image showing a couple Dancing together to music in the living room": "neutral",
    "image showing a couple Cooking a meal together in the kitchen": "neutral",
    "image showing a couple Going for a walk or hike in the park or countryside": "neutral",
    "image showing a couple Watching a movie or TV show together on the couch": "neutral",
    "image showing a couple Going out for a romantic dinner at a restaurant": "neutral",
    "image showing a couple Having a picnic in a scenic location": "neutral",
    "image showing a couple Going for a bike ride or cycle together": "neutral",
    "image showing a couple Relaxing in a hot tub or spa together.": "neutral",
    "image showing a man and a woman sleeping": "neutral",
    "image showing a couple sitting very close to each other, indicating their affection": "neutral",
    "image showing couple's close faces, suggesting they are looking each other out of affection": "neutral",
    "image showing a man and a girl sitting side by side with a pen in hand": "neutral",
    "image showing friends sitting in the restaurant and talking": "neutral",
    "image showing friends enjoying themselves": "neutral",
    "image showing a couple engaged in a daily routine activity": "neutral",
    "image showing couple engaged in a discussion": "neutral",
    # Solo activities (male, female)
    "image showing a close-up image of a person's face with clear facial features": "neutral",
    "image showing a person standing in a crowd or group": "neutral",
    "image showing a a person in motion or engaged in an activity, such as running or playing a sport": "neutral",
    "image showing a person Reading a book in a quiet place": "neutral",
    "image showing a person Running on a treadmill at the gym": "neutral",
    "image showing a person Painting a landscape on a canvas": "neutral",
    "image showing a person Playing a game of chess against a computer": "neutral",
    "image showing a person Writing in a journal or diary": "neutral",
    "image showing a person Meditating in a calm and quiet space": "neutral",
    "image showing a person Practicing yoga poses at home": "neutral",
    "image showing a person Playing a musical instrument": "neutral",
    "image showing a person Gardening and tending to plants in the backyard": "neutral",
    "image showing a person Cleaning and organizing a closet": "neutral",
    "image showing a person Cooking a meal from scratch in the kitchen": "neutral",
    "image showing a person Sewing and creating clothing or accessories": "neutral",
    "image showing a person Doing a home workout routine with weights or bodyweight exercises": "neutral",
    "image showing a person Taking a relaxing bath with candles and music": "neutral",
    "image showing a person Sketching and drawing with charcoal or pencil": "neutral",
    "image showing a person Playing video games on a console or computer": "neutral",
    "image showing a person watching a movie or TV show on a couch": "neutral",
    "image showing a person clicking photo using the camera": "neutral",
    "image showing a woman dressed in a bra and panties or lingerie": "neutral",
    "image showing a person wearing clothing specifically designed for swimming or water activities": "neutral",
    "image showing a person shouting out loud": "neutral",
    "image showing a woman in a bikini covering her fully": "neutral",
    "image showing a woman in a red or pink dress": "neutral",
    "image showing empty handed ancient soldier": "neutral",
    # People Activities:
    "image showing people Playing a game of basketball or soccer at the park with friends": "neutral",
    "image showing people Having a picnic or barbecue with family and friends": "neutral",
    "image showing people Going on a group hike or nature walk": "neutral",
    "image showing people Participating in a team-building activity at work or school": "neutral",
    "image showing people Playing a board game or card game with a group of friends": "neutral",
    "image showing people Attending a music festival or concert with a group of people": "neutral",
    "image showing people Going to a theme park or amusement park with friends or family": "neutral",
    "image showing people Taking a group fitness class at a gym or studio": "neutral",
    "image showing people Going on a group camping trip or weekend getaway": "neutral",
    "image showing people Participating in a volunteer activity or community service project": "neutral",
    "image showing people Attending a sporting event with a group of friends": "neutral",
    "image showing people Participating in a book club or discussion group": "neutral",
    "image showing people Going to a trivia night or pub quiz with friends": "neutral",
    "image showing people Attending a theater performance or musical with a group of people": "neutral",
    "image showing people Going on a group bike ride or cycle tour": "neutral",
    "image showing people engaged in a discussion": "neutral",
    "image showing people engaged in a particular sports activity": "neutral",
    "image showing discussion between group of people": "neutral",
    "image showing people or couple hugging each other": "neutral",
    "image showing couple or people sleeping or resting": "neutral",
    "image showing people sleeping in the dark or at night": "neutral",
    "image showing people walking": "neutral",
    "image showing people gesturing": "neutral",
    "image showing joyful people dancing, enjoying themselves": "neutral",
    "image showing people inside a store or shop": "neutral",
    "image showing people in a park or outdoor setting": "neutral",
    "image showing people in a healthcare or professional setting": "neutral",
    "image showing people exercising": "neutral",
    "image showing people using public transportation": "neutral",
    "image showing people holding a child or baby": "neutral",
    "image showing people enjoying themselves on the street": "neutral",
    "image showing people performing a work-related activity": "neutral",
    "image showing people performing a leisure or relaxation activity": "neutral",
    "image showing people performing a everyday or routine activity": "neutral",
    "image showing people playing musical instruments and singing": "neutral",
    "image showing people sitting around and enjoying their company": "neutral",
    "image showing conversation between people on a street corner": "neutral",
    "image showing people in bikini and underwear enjoying themselves": "neutral",
    "image showing soldiers marching or walking or standing": "neutral",
    "image depicting groups of the crowd on the field": "neutral",
    # Expressions:
    "image showing an expression of Smiling with teeth showing": "neutral",
    "image showing an expression of Frowning with eyebrows furrowed": "neutral",
    "image showing an expression of Raising eyebrows in surprise": "neutral",
    "image showing an expression of Puckering lips in a kiss or whistle": "neutral",
    "image showing an expression of Sticking out tongue in jest or playfulness": "neutral",
    "image showing an expression of Squinting eyes in suspicion or skepticism": "neutral",
    "image showing an expression of Rolling eyes in annoyance or exasperation": "neutral",
    "image showing an expression of Nodding head in agreement or understanding": "neutral",
    "image showing an expression of Shaking head in disagreement or disbelief": "neutral",
    "image showing an expression of Wincing or grimacing in pain or discomfort": "neutral",
    "image showing an expression of Blinking rapidly in surprise or fear": "neutral",
    "image showing an expression of Scrunching nose in distaste or disgust": "neutral",
    "image showing an expression of Pouting lips in disappointment or dissatisfaction": "neutral",
    "image showing an expression of Raising eyebrows and widening eyes in curiosity or interest": "neutral",
    "image showing an expression of Smirking or smugly smiling with one corner of the mouth lifted": "neutral",
    "image showing an expression of Biting lip in nervousness or anticipation": "neutral",
    "image showing an expression of Licking lips in hunger or desire": "neutral",
    "image showing an expression of Dropping jaw in shock or amazement": "neutral",
    "image showing an expression of Clenching jaw in anger or frustration": "neutral",
    "image showing an expression of Tilting head to the side in confusion or thoughtfulness": "neutral",
    "image depicting affection in a friendly or familial context": "neutral",
    # Settings:
    "image showing an outdoor beach setting with sand, water, and palm trees": "neutral",
    "image showing an indoor kitchen setting with appliances, countertops, and cabinets": "neutral",
    "image showing an outdoor park setting with grass, trees, and benches": "neutral",
    "image showing an indoor living room setting with furniture, decorations, and a television": "neutral",
    "image showing an outdoor mountain setting with rocks, trees, and a clear blue sky": "neutral",
    "image showing an indoor bedroom setting with a bed, pillows, and a nightstand": "neutral",
    "image showing an outdoor urban setting with buildings, streets, and people": "neutral",
    "image showing an indoor dining room setting with a table, chairs, and place settings": "neutral",
    "image showing an outdoor forest setting with trees, foliage, and a dirt path": "neutral",
    "image showing an indoor bathroom setting with a sink, toilet, and shower/tub": "neutral",
    "image showing an outdoor desert setting with sand, rocks, and cacti": "neutral",
    "image showing an indoor office setting with desks, chairs, and computers": "neutral",
    "image showing an outdoor countryside setting with fields, barns, and farm animals": "neutral",
    "image showing an indoor gym/fitness setting with equipment, weights, and exercise mats": "neutral",
    "image showing an outdoor garden setting with flowers, plants, and a pond/fountain": "neutral",
    "image showing an indoor library setting with shelves, books, and reading tables": "neutral",
    "image showing an outdoor river or lake setting with water, trees, and boats": "neutral",
    "image showing an indoor theater setting with a stage, seating, and lighting": "neutral",
    "image showing an outdoor cityscape setting with skyscrapers, traffic, and lights": "neutral",
    "image showing an indoor museum setting with exhibits, displays, and artifacts.": "neutral",
    "image showing a snowy or icy environment or icy floor": "neutral",
    "image showing a traffic scene or highway": "neutral",
    "image showing a building interior or interior design": "neutral",
    "image showing a neutral outdoor scene or natural setting": "neutral",
    "image showing fire around": "neutral",
    "image showing plants and leaves": "neutral",
    "image showing a long shot of an environment": "neutral",
    # neutral activities
    "image showing two people engaged in a heated argument, shouting at each other": "neutral",
    "image showing a person running through a crowded street, with people and cars passing by": "neutral",
    "image showing a person jumping from a high ledge and landing safely on the ground": "neutral",
    "image showing a person breaking a window with a rock and climbing inside a building": "neutral",
    "image showing a person crying and hugging another person in a moment of emotional release": "neutral",
    "image showing a car chase scene with two vehicles weaving in and out of traffic": "neutral",
    "image showing a person sneaking through a dark alleyway while trying to avoid detection.": "neutral",
    "image showing a person performing an intricate dance routine with a group of dancers": "neutral",
    "image showing a person delivering an impassioned speech to a large crowd": "neutral",
    "image showing a person engaged in a high-speed chase on a motorcycle.": "neutral",
    "image showing a person using advanced technology to hack into a computer system.": "neutral",
    "image showing a person sitting quietly and contemplating something in deep thought": "neutral",
    "image showing a person running through a dense forest or jungle": "neutral",
    "image showing a person going on a long, arduous hike in the wilderness": "neutral",
    "image showing a person teaching a group of people how to dance": "neutral",
    "image showing a person participating in a dangerous extreme sport, such as skydiving or snowboarding": "neutral",
    "image showing a person performing a stand-up comedy routine in front of a live audience": "neutral",
    "image showing a person engaging in a romantic and passionate dance with another person": "neutral",
    "image showing a person participating in a tense and competitive game of chess": "neutral",
    "image showing a person learning how to surf in the ocean": "neutral",
    "image showing a person participating in a high-stakes horse race or car race": "neutral",
    "image showing a person attempting climbing the mountain": "neutral",
    "image showing a person engaging in a fast-paced game of basketball or other sports": "neutral",
    "image showing a person practicing for a high-profile and difficult musical performance": "neutral",
    "image showing a person training for a high-pressure and dangerous job, such as a firefighter or astronaut": "neutral",

    # guns
    "image showing gun wall rack with rifles and pistol": "violence",
    "image showing armed with rifles or shotgun": "violence",
    "image showing handheld pistol": "violence",
    "image showing a revolver brandishing at the corner": "violence",
    "image showing close-up of holding a revolver or pistol": "violence",
    "image showing armed robbers carrying a pistol to rob the bank, the money": "violence",
    "image showing holding or brandishing a pistol or a revolver": "violence",
    "image showing holding or brandishing a shotgun": "violence",
    "image showing holding or brandishing a Rifle or Carbine or Assault rifle": "violence",
    "image showing holding or brandishing a Machine gun": "violence",
    "image showing a case of revolver or shotgun": "violence",
    "image showing a long shot of a rifle or a shotgun": "violence",
    "image showing a pistol or a revolver kept on the surface": "violence",
    "image showing a Rifle or Carbine or Assault rifle kept on the surface": "violence",
    "image showing a Machine gun kept on the surface": "violence",
    "image showing different kind of gun models kept for display": "violence",
    "image showing soldiers armed with gun weapons": "violence",
    "image showing different kind of guns laid out on the table for firearms trafficking, firearms on the table": "violence",
    "image showing the assembly or production of gun weapons": "violence",
    "image showing collection of rifles and carbines, various firearms hang on special mounts on the wall in the background": "violence",
    "image showing a man firing the bullets from a pistol or a revolver": "violence",

    # sword
    "image showing foil fencing": "violence",
    "image showing collection of swords kept in the cabinet": "violence",
    "image showing holding or brandishing a knife": "violence",
    "image showing holding or brandishing a dagger": "violence",
    "image showing holding or brandishing a double-edged two-handed sword": "violence",
    "image showing holding or brandishing a throwing spears and javelins": "violence",
    "image showing a knife kept on the surface": "violence",
    "image showing close-up of holding a knife": "violence",
    "image showing different kind of bladed weapons like knives, swords kept for display": "violence",
    "image showing holding or brandishing an sharp ax": "violence",
    "image showing stabbed knife": "violence",
    "image showing stabbing knife in the chest": "violence",
    "image showing a sword fight, clanging blades and dodging": "violence",
    "image showing different kind of swords kept on the surface": "violence",
    "image showing a polearms with spikes and hammers": "violence",
    "image showing a knife attack": "violence",
    "image showing a ancient soldier holding a sword": "neutral",

    # war artillery
    "image showing military equipment on the ground": "violence",
    "image showing tanks or bulldozers on the ground": "violence",
    "image showing holding bombs or grenades": "violence",
    "image showing bombs or grenades or bullets kept on the surface": "violence",
    "image showing Cannons": "violence",
    "image showing naval artillery": "violence",
    "image showing heavy artillery": "violence",
    "image showing holding or brandishing blunt staves or bow and arrow": "violence",
    "image showing the soldiers on the turret of the tank climb into the tank": "violence",
    "image showing soldiers are holding rifles near a military tank": "violence",
    "image showing display of artillery": "violence",
    "image showing a submarine": "violence",
    # stick
    "image depicting stick assault": "violence",
    "image showing beating attackers using a baseball stick": "violence",
    "image showing stick fighting": "violence",
    # blood_gore
    "image showing a pool of blood on the ground": "violence",
    "image showing a bloody hand print on a wall": "violence",
    "image showing a Blood spattered on a knife or sword": "violence",
    "image showing a dismembered body part": "violence",
    "image showing blood oozing from a wound": "violence",
    "image showing a severed limb lying on the ground": "violence",
    "image showing a bloody knife or blade": "violence",
    "image showing a bloody crime scene with multiple victims": "violence",
    "image showing a bloodied face or head": "violence",
    "image showing a graphic and bloody injury": "violence",
    "image showing a wound with visible bone or muscle tissue": "violence",
    "image showing a bloody trail leading away from the scene of a crime": "violence",
    "image showing a bloody hand holding a knife": "violence",
    "image showing a bloody hand holding a gun": "violence",
    "image showing a person covered in blood and gore": "violence",
    "image showing a gory and gruesome murder scene": "violence",
    "image showing a bloody and disfigured corpse": "violence",
    "image showing a decapitated head with visible wounds and blood": "violence",
    "image showing Blood spraying from a wound due to impact or pressure": "violence",
    "image showing a bloody and mangled body from a vehicular accident": "violence",
    "image showing a bloody and torn piece of clothing": "violence",
    "image showing Blood spattered on the walls, floor, and ceiling": "violence",
    "image showing a bloody and mutilated animal carcass": "violence",
    "image showing a wound with exposed organs or internal body parts": "violence",
    "image showing a bloodied knife with visible tissue and bone fragments": "violence",
    "image showing a person covered in blood and wounds after a violent assault": "violence",
    "image showing Blood pouring from a body onto the ground": "violence",
    "image showing a victim with visible cuts, bruises, and lacerations": "violence",
    "image showing a bloody and disfigured face or head": "violence",
    "image showing a bloody hand reaching out for help": "violence",
    "image showing a person covered in blood and gore from a zombie or monster attack": "violence",
    "image showing Blood smeared on the walls or other surfaces in a pattern": "violence",
    "image showing a person with a bloody and distorted face due to torture or mutilation": "violence",
    "image showing a bloody and shattered glass surface": "violence",
    "image showing a bloody and broken bone protruding from a wound": "violence",
    "image showing a person lying in a pool of blood with multiple injuries": "violence",
    "image showing a bruise on a person's arm or leg": "violence",
    "image showing a cut or laceration on the skin": "violence",
    "image showing a person with a black eye or swollen face": "violence",
    "image showing a bandaged or wrapped wound on a body part": "violence",
    "image showing a person with a visible cast on their arm or leg": "violence",
    "image showing a person with a visible splint on their finger or hand": "violence",
    "image showing a person with visible stitches or sutures on their skin": "violence",
    "image showing a person with a noticeable scar on their face or body": "violence",
    "image showing a person with a swollen or sprained ankle": "violence",
    "image showing a person with a visible burn or blister on their skin": "violence",
    "image showing a motionless body lying on the ground": "violence",
    "image showing a body that has turned blue or purple due to lack of oxygen": "violence",
    "image showing a body with missing limbs or appendages": "violence",
    "image showing a body with visible puncture wounds or stabs": "violence",
    "image showing a body with missing or disfigured facial features": "violence",
    "image showing a body with visible signs of torture or mutilation": "violence",
    "image showing a body with missing or removed organs": "violence",
    "image showing a body with visible signs of burns or scalding": "violence",
    "image showing a body with exposed bone or tissue": "violence",
    "image showing a body with deep and visible cuts or slices": "violence",
    "image showing a body with ripped or shredded flesh": "violence",
    "image showing a body with crushed or mangled body parts": "violence",
    "image showing human organs in a jar": "violence",
    "image showing any bloodstains or lots of blood on the ground or other surfaces": "violence",
    "image showing people or animals bleeding or hemorrhaging": "violence",
    "image depicting any acts of physical violence, such as punching, kicking, or choking, bleeding, injury shown": "violence",
    "image depicting blood or gore in the context of animal abuse or assault or hunting": "violence",
    "image showing a person with rotted or decomposed face with gore": "violence",
    "image showing a rotted or decomposed animal with gore all over the body": "violence",
    "image showing a burned or charred body beyond recognition": "violence",
    "image showing a burned body, burning": "violence",
    "image showing a person is burning surrounded by a group of people": "violence",
    "image showing a person is cutting the human head using a sword": "violence",
    "image showing a person around a mutilated body": "violence",
    "image showing a person around the corpse": "violence",
    "image showing a bloody dismemberment body": "violence",
    "image showing a injured person among the lifeless human bodies": "violence",
    "image showing a injured person with significant blood loss and clothes are covered with blood": "violence",
    "image showing many dead bodies on the ground after a brutal fight": "violence",
    "image showing a dead mutilated corpse": "violence",
    "image showing a dead body or corpse": "violence",
    # violent acts:
    "image showing the victims brutally assaulted, indicating violent act": "violence",
    "image showing the victim tied behind their backs or strapped their head and legs restrained, indicating violent act of executions": "violence",
    "image depicting impaled heads on spear brutal execution methods": "violence",
    "image showing a person in the guillotine, Executioner and axe": "violence",
    "image showing a person with tied up and locked with rope": "violence",
    # Whole frame have violence seen
    "image showing aftermath of violent act": "violence",
    "image showing violent acts of bombings or explosives, indicating explosion aftermath": "violence",
    "image depicting a bomb explodes in a crowded marketplace, causing chaos and destruction": "violence",
    # fighting
    "image showing group of people engage in a brutal fistfight, kicking and punching each other on the ground": "violence",
    # *prescriptions shown
    "image showing a prescription bottle": "drugs",
    "image showing a doctor holding glass bottle with medicine, prescription": "drugs",
    "image showing a doctor, researcher or scientist holding vaccine and syringe with needle for vaccination": "drugs",
    # *marijuana shown
    "image showing marijuana joint, rolling marijuana cannabis joint": "drugs",
    "image showing marijuana joints kept on the surface": "drugs",
    "image showing a bag or packets of marijuana, weed": "drugs",
    # *psychedelic shown
    "image showing LSD pills kept on the surface": "drugs",
    # *hard drugs shown
    "image showing pile of illegal drug cocaine bags or packages visible": "violence",
    "image showing the pile of illegal drug packets kept in a shelf": "drugs",
    "image showing white powder substance, cocaine powder lines on a surface": "drugs",
    "image showing lighter to heat up a substance, heroin on a spoon": "drugs",
    "image showing drug syringe or needle, drug paraphernalia kept on the surface": "drugs",
    "image showing preparing dose of crack cocaine on street": "drugs",
    "image showing white powder substance packet, lines and pile of cocaine on black background": "drugs",
    "image showing cocaine with rows, rows and lines of powder and syringe, with a drool blade beside it": "drugs",
    "image showing cocaine powder cut with card on mirror, hand dividing white powder narcotic": "drugs",
    "image showing discarded syringe on waste ground": "drugs",
    "image showing a person holding a syringe and needle, focus on syringe and needle": "drugs",
    "image depicting a person holding a cocaine powder substance in a packet or pouch": "drugs",
    "image showing the pile of discarded needles and narcotics drug paraphernalia scattered on the surface": "drugs",
    # manufacturing
    "image showing packing illegal drugs in a packet, indicating drug manufacturing assembly": "drugs",
    "image showing Shot of Sterile Pharmaceutical Manufacturing Laboratory": "drugs",
    "image showing pharmacy medicine capsule pill in production line at medical factory": "drugs",
    "image showing underground Drug Laboratory, Chemists Wearing Protective Masks and Coveralls Mix Chemicals": "drugs",
    "image showing drug packets in a drug manufacturing set up": "drugs",
    "image showing illegal drug manufacturing, Handling of drugs, drug treatment": "drugs",
    "image showing cannabis plantation, bushes of cannabis by a stern plant, leaves of narcotic plants for the manufacture of hemp, marijuana": "drugs",
    "image showing cocaine warehouse, plastic bags with cocaine in a box": "drugs",
    "image showing Cocaine packets being weighted Drugs are being seized by people from cartels infinite production": "drugs",
    "image showing a person wearing protective gear and handling hazardous substances": "drugs",
    "image showing a laboratory or workshop containing various chemicals, glassware, and equipment": "drugs",
    "image showing a container or vessel used for mixing or processing drugs": "drugs",
    "image showing a person using a heat source to boil or evaporate a liquid": "drugs",
    "image showing a person measuring and weighing precise amounts of chemicals or substances": "drugs",
    "image showing a person using a press or machine to compress powders into pills or tablets": "drugs",
    "image showing a person packaging or labeling finished drug products": "drugs",
    "image showing a person transporting large quantities of drugs or drug-making materials": "drugs",
    "image showing a chemical process to extract opium or morphine from poppy plants": "drugs",
    "image showing a person using a large-scale distillation apparatus to purify drugs such as methamphetamine or fentanyl": "drugs",
    "image showing a person using a press or machine to compress powders into hard drug forms like crack cocaine or heroin": "drugs",
    "image showing a person using a pill press machine to manufacture counterfeit prescription drugs": "drugs",
    "image showing a person using a sophisticated laboratory or industrial-grade equipment to manufacture hard drugs": "drugs",
    # *injecting shown
    "image showing a close-up shot shows a needle piercing the skin and injecting a liquid substance into the body": "drugs",
    "image showing injecting heroin in the body with syringe or needle": "drugs",
    "image showing a drugs addict injecting heroin in his arm": "drugs",
    "image showing a doctor injecting a patient with syringe to collect blood sample": "drugs",
    "image showing nurse with the syringe injects the vaccine to the patient lying in the hospital room": "drugs",
    # *smoking shown
    "image showing a person smoking marijuana (weed, cannabis)": "drugs",
    "image showing marijuana, weed smoke from bongs": "drugs",
    "image showing group of people smoking cannabis or marijuana joints": "drugs",
    "image showing a guy smoking marijuana joint while relaxing": "smoking",
    # *snorting shown
    "image showing a person inhaling a substance through their nostrils, often making a snorting sound as they do so.": "drugs",
    "image depicting a line of white powder is shown being snorted off a flat surface": "drugs",
    "image showing a addict is snorting cocaine powder with rolled banknote": "drugs",
    "image showing snorting of cocaine powder lines": "drugs",
    "image showing a drug addict snorting and sniffing cocaine lines on mirror with rolled banknote": "drugs",
    # *oral shown
    "image showing drug pills on tongue": "drugs",
    "image showing a person taking daily dose of a prescription pill": "drugs",
    "image showing sick ill person hold two pills on hand pouring capsules from medication bottle take medicine": "drugs",
    # selling shown
    "image showing narcotics dealer drug holding bag with cocaine drug powder, cocaine in plastic packet": "drugs",
    "image showing mafia narcotics drug deal, narcotics hard drugs shown around": "drugs",
    "image showing a holding a cocaine drug powder pouch and selling": "drugs",
    "image showing drug selling, drug pouch in hand, holding cocaine packet": "drugs",
    # tobacco shown
    "image showing a person lights a cigarette for smoking": "smoking",
    "image showing close up of holding a cigarette in hand": "smoking",
    "image showing individuals holding a lit cigarette": "smoking",
    "image showing any depictions of tobacco products, including cigarettes or cigars": "smoking",
    "image showing whole lots of cigarettes or cigarette packet kept on the surface": "smoking",
    "image showing several ashtrays filled to the brim with cigarette butts": "smoking",
    # tobacco smoking and shown
    "image showing individuals who appear to be smoking, even if the cigarette is not clearly visible": "smoking",
    "image showing a person smoking cigarette near people smelling pollution, passive smoking concept": "smoking",
    "image showing a person holding smoking a cigarette in hand, cigarette smoke spread": "smoking",
    "image showing a individuals smoking a cigarette on the street": "smoking",
    "image showing group of people smoking cigarettes together": "smoking",
    "image showing couple smoking cigarette": "smoking",
    "image showing side view of a person sitting and blowing smoke": "smoking",
    "image showing inhaling of a vapor with electronic cigarette, vape flavor liquid chemicals, use of e-cigarettes": "smoking",
    "image showing a tobacco user smoking cigarette": "smoking",
    "image showing a person enjoying smoking cigarette, focus on the cigarette, holing cigarette in hand": "smoking",
    # alcohol Shown
    "image showing beer bottle kept on the surface": "alcohol",
    "image showing beer glass or mug filled with beer kept on the surface": "alcohol",
    "image showing whiskey or vodka bottle kept on the surface": "alcohol",
    "image showing a glass filled with whiskey or vodka kept on the surface": "alcohol",
    "image showing holding a beer bottle": "alcohol",
    "image showing holding a whiskey bottle": "alcohol",
    "image showing holding a glass of whiskey": "alcohol",
    "image showing holding a beer mug": "alcohol",
    "image showing empty bottles of whiskey or beer": "alcohol",
    "image showing a person holding a beer bottle at the corner of the frame": "alcohol",
    "image showing Set of strong alcoholic drinks like vodka, rum, cognac, tequila, brandy and whiskey": "alcohol",
    "image showing shelves or cabinet with bottles of alcohol in background": "alcohol",
    "image showing a cabinet filled with beer bottles": "alcohol",
    "image showing a close up of holding crystal glass of whiskey or brandy": "alcohol",
    "image showing bottles of wine in supermarket": "alcohol",
    "image showing beer cans kept on the table": "alcohol",
    "image showing alcohol cabinets and shelving for liquor storage at the bar": "alcohol",
    "image depicting implied alcohol use": "alcohol",
    "image showing side view of a person sitting with alcohol bottle of beverage": "alcohol",
    # alcohol Shown and consumption:
    "image showing people sit around a table and drinking beer or whiskey": "alcohol",
    "image showing a individual drinking whiskey": "alcohol",
    "image showing a individual drinking beer": "alcohol",
    "image showing a anonymous alcoholic person drinking behind bottles of alcohol": "alcohol",
    "image showing group of people drinking alcohol while discussing": "alcohol",
    "image showing group of party people, men and women, drinking beer in a pub or bar": "alcohol",
    "image showing a crowded bar, showing patrons drinking and laughing, with glasses and bottles of alcohol on every table": "alcohol",
    "image showing a romantic couple drinking glass of wine Together": "alcohol",
    "image depicting people consuming alcohol beverages or alcoholic drinks": "alcohol",
    "image depicting people sitting around and enjoying drinking beer or vodka or whiskey together": "alcohol",
    "image showing a woman drinking wine or whiskey with a bottle or a glass": "alcohol",
    "image showing a man drinking wine or whiskey or beer with a bottle or a glass": "alcohol",
    "image showing a woman having dinner and a glass of red wine with a person": "alcohol",
    "image showing group of people having party, smoking cigarette and drinking beer": "smoking,alcohol",
    "image showing a person drinking whiskey or beer at the counter in a bar": "alcohol",
    # bdsm
    "image depicting erotic practices of engaging in fetishistic behaviors, such as bondage, BDSM, or other kinks": "sexuality",
    "image showing set of erotic toys for BDSM, the game of sexual slavery with a whip, gag and leather blindfold": "sexuality",
    "image showing a girl is tied up using BDSM toys for sexual pleasure": "sexuality",
    # bestiality
    "image depicting sexual acts that is considered as bestiality": "sexuality",
    # group sex
    "image showing sexual act of threesome or foursome": "sexuality",
    "image depicting group sex or orgy": "sexuality",
    "image showing group of men and group of women enjoying and engaged in a sexual activity of group sex": "sexuality",
    # kissing
    "image showing kissing or smooching or romantic lip-lock": "sexuality",
    "image showing an intimate couple is making out and kissing, lip-lock": "sexuality",
    # making out
    "image showing a romantic couple, man and woman making love": "sexuality",
    "image showing a romantic sexy couple, man and woman having a foreplay": "sexuality",
    "image showing sexually suggestive poses or gestures, such as individuals touching or caressing another person's body": "sexuality",
    "image depicting beautiful intimate couple making love, couple having sex": "sexuality",
    "image depicting a Almost stripped man and woman in a loving embrace": "sexuality",
    "image showing a Passionate couple enjoying sensual foreplay before having sex or making love, aroused lover holding hand on butt of sexy hot beautiful lady on top teasing a man": "sexuality",
    # masturbation
    "image showing a man is masturbating, touching his penis for sexual pleasure, lust": "sexuality",
    "image showing sexy woman in a underwear, touching herself, lust": "sexuality",
    "image showing a sexy girl playing, touching with herself, orgasm, cuddling herself": "sexuality",
    # necrophilia
    "image depicting necrophilia, a type of sexual attraction towards or a sexual act involving corpses": "sexuality",
    # oral sex
    "image showing woman touching groin area, preliminary caresses and oral sex, fellatio in action": "sexuality",
    "image showing a man touching the groin area into the naked legs of a woman": "sexuality",
    "image showing a man touching the naked legs of the woman": "sexuality",
    "image showing man hands on woman's ass, preliminary caresses and oral sex, Cunnilingus and pleasure": "sexuality",
    # sex toy
    "image showing sex toy's like dildos or vibrators": "sexuality",
    "image showing a sex toy shop": "sexuality",
    "image showing holding a dildo or vibrator, sex toy": "sexuality",
    # vaginal
    "image depicting a sexual position, sex poses illustration of man and woman, vaginal sex position": "sexuality",
    "image showing attractive sexy couple having sexual intercourse, sexual act of vaginal sex or penetrative sex": "sexuality",
    "cropped image of beautiful passionate couple having vaginal sex, sexual intercourse": "sexuality",
    "image depicting a close up shot of a sexual position, vaginal intercourse": "sexuality",
    # same sex
    "image showing a lesbian couple (two girls) kissing or smooching or romantic lip-lock": "sexuality",
    "image showing a gay couple (two men) kissing or smooching or romantic lip-lock": "sexuality",
    "image showing a gay couple (two men) having anal sex": "sexuality",
    "image showing a lesbian couple (two girls) making out": "sexuality",
    "image showing a gay couple (two men) making out": "sexuality",
    # front_female
    "image showing a upper naked body of a female, with focus on breast or boobs": "nudity",
    "image showing a nude woman, exposing breast or boobs": "nudity",
    "image showing a girl striping exposing her breast or boobs": "nudity",
    "image showing close up shot of a girl's breast or boobs": "nudity",
    "image showing a naked woman sitting, breast and nipples visible": "nudity",
    "image showing a girl exposing her nipples, having sex": "nudity",
    # front_male
    "image showing upper naked body of a man, with focus on bare chest and nipples": "nudity",
    "image showing a naked man sitting, chest visible": "nudity",
    "image showing a nude man, exposing chest and nipples": "nudity",
    "image showing naked man enjoying and smoking cigarettes": "smoking,nudity",
    # full_female
    "image showing a female's full naked frontal body including breast, nipples and vagina or vulva": "nudity",
    "image showing a side view of a completely naked woman, full nudity": "nudity",
    "image showing a completely naked gorgeous seductive woman lying on bed, full nudity": "nudity",
    # full_male
    "image showing a male's full naked frontal body including chest, nipples and penis or dick, full nudity": "nudity",
    "image showing a side view of a completely naked man, full nudity": "nudity",
    # genitals_displayed
    "image showing naked baby showing genitals": "nudity",
    "image showing clearly visible exposed genitals": "nudity",
    "image showing close-up shot of genitals, vagina or penis": "nudity",
    # rear_view
    "image showing a female full naked back body including bare back and ass or buttocks": "nudity",
    "image showing a male full naked back body including bare back and ass or buttocks": "nudity",
    "image showing the lower naked body, with focus on buttocks or ass": "nudity",
    "image showing the close up shot of butt crack or ass": "nudity",
    # all views / couple
    "image depicting groups of the fully naked crowd on the field": "nudity",
    "image showing a completely naked man and a completely naked woman sleeping or sitting next to each other": "nudity",
    "image depicting naked people enjoying themselves exposing their private body parts": "nudity",
    "image showing a woman dressed in a revealing clothes or exposing dress and her private parts are visible from the dress": "nudity",
    "image showing a side view of the completely naked couple holding, caressing each other": "sexuality,nudity",
    "image showing a woman with a breast and a man with bare chest": "nudity",
    # *painting
    "image depicting a painting of fully naked body including breast and nipples and genitals": "nudity",
    # *blowup dolls
    "image depicting a naked blow up doll, a type of anthropomorphic sex toy": "nudity",
    # *statue
    "image depicting sculptures that resemble genitalia or other sexualized body parts": "nudity",
    "image depicting statues that depict nudity or intimate body parts": "nudity",
    # *mannequin
    "image depicting mannequins that are shown without clothing or that appear to be displaying intimate body parts": "nudity",
    # profanity
    "image depicting gestures using the middle finger": "profanity",
    "image depicting the middle finger being used in combination with profanity": "profanity",
}


In [11]:
folder_path =  "/content/gdrive/MyDrive/violence/"
list_of_images = []
for image_path in glob.glob(folder_path + "*"):
  list_of_images.append(image_path)
list_of_images.sort(key=natural_keys)
for image_path in list_of_images:
  res = get_prediction(image_path, list(first_level.keys()), how_many_predictions)

1_img_000001.png
image showing a person covered in blood and gore from a zombie or monster attack 35.06
image showing a person sneaking through a dark alleyway while trying to avoid detection. 17.81
image showing any bloodstains or lots of blood on the ground or other surfaces 2.98
___________________________
1_img_000002.png
image showing a person covered in blood and gore from a zombie or monster attack 30.46
image showing a person with rotted or decomposed face with gore 6.92
image showing a person covered in blood and gore 4.63
___________________________


KeyboardInterrupt: ignored